In [5]:
from datasets import *

In [6]:
def test(model, enc_input, start_symbol):
    # Starting Reference: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    enc_outputs, enc_self_attns = model.Encoder(enc_input)
    dec_input = torch.zeros(1, tgt_len).type_as(enc_input.data)
    next_symbol = start_symbol
    for i in range(0, tgt_len):
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.Decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input


In [15]:
enc_inputs, dec_inputs, dec_outputs = make_data()
loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 21, True)
enc_inputs, _, _ = next(iter(loader))
model = torch.load('model.pth')

results=[]
for k in range(20):
    predict_dec_input = test(model, enc_inputs[k].view(1, -1).cuda(), start_symbol=tgt_vocab["S"])
    predict, _, _, _ = model(enc_inputs[k].view(1, -1).cuda(), predict_dec_input)
    predict = predict.data.max(1, keepdim=True)[1]

    input=''
    for i in enc_inputs[k]:
        if i == 0:
            pass
        else:
            input+=src_idx2word[int(i)]
    
    print(input, '->',[idx2word[n.item()] for n in predict.squeeze()])
    results.append([input]+[idx2word[n.item()] for n in predict.squeeze()])

中央扶手放倒 -> ['空调系统', '后排空调', '后排空调', '后排空调']
驾驶员遮阳板化妆镜 -> ['化妆镜', '化妆镜', '化妆镜', '化妆镜']
后排中央扶手带杯架 -> ['后排空调', '座椅', '座椅', '座椅']
车顶衣挂 -> ['座椅', '空调系统', '空调系统', '空调系统']
车顶眼镜盒 -> ['化妆镜', '座椅', '座椅', '座椅']
副驾驶员座椅腿肚支撑 -> ['座椅', '化妆镜', '化妆镜', '化妆镜']
前排中央扶手 -> ['座椅', '座椅', '座椅', '座椅']
前排杯架 -> ['后排空调', '座椅', '座椅', '座椅']
手套箱带照明 -> ['空调系统', '座椅', '座椅', '座椅']
后排空调出风口 -> ['后排空调', '后排空调', '后排空调', '后排空调']
后排杯架 -> ['后排空调', '后排空调', '后排空调', '后排空调']
前排座椅靠背文件袋 -> ['后排空调', '后排空调', '后排空调', '后排空调']
后排功能控制-控制音响 -> ['E', '空调系统', '空调系统', '空调系统']
带密码的手套箱 -> ['化妆镜', '空调系统', '空调系统', '空调系统']
后排功能控制-控制空调 -> ['空调系统', '空调系统', '空调系统', '空调系统']
副驾遮阳板化妆镜 -> ['化妆镜', '化妆镜', '化妆镜', '化妆镜']
后排中央扶手 -> ['后排空调', '座椅', '座椅', '座椅']
仿皮座椅材质 -> ['后排空调', '后排空调', '后排空调', '后排空调']
车控车设门盖手套箱 -> ['后排空调', '空调系统', '空调系统', '空调系统']
手动调节第二排头枕 -> ['后排空调', '化妆镜', '化妆镜', '化妆镜']


In [17]:
import pandas as pd
pd.DataFrame(results).to_csv('./results.csv')